In [1]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # to enable deterministic behavior with CuBLAS
# NOTE: to avoid error
#   RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` 
#   or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it
#   uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an 
#   environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or 
#   CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to 
#   https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility

from transformers import set_seed
set_seed(42, deterministic=True) # for reproducibility

In [2]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

## Single-label classification

In [3]:
# load a example classification dataset from hf hub
from datasets import load_dataset
train_dataset = load_dataset("ag_news", split="train[:600]")
val_dataset = load_dataset("ag_news", split="test[:200]")
test_dataset = load_dataset("ag_news", split="test[200:400]")

num_classes = len(set(train_dataset["label"]))

In [4]:
from transformers.trainer_utils import PredictionOutput
from sklearn.metrics import f1_score
def compute_metrics(p: PredictionOutput):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = np.sum(preds == labels) / len(labels)
    f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "macro_f1": f1}

### using the `SetfitModel`'s `fit()` method

In [ ]:
from sentence_transformers import SentenceTransformer
from setfit.modeling import SetFitHead
# from src.finetuning.setfit_extensions.class_weights_head import compute_class_weights, SetFitHeadWithClassWeights
from src.finetuning.setfit_extensions.early_stopping import (
    SetFitModelWithEarlyStopping,
    EarlyStoppingTrainingArguments
)

In [ ]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
body = SentenceTransformer(model_id, model_kwargs={"device_map": "auto"})

head = SetFitHead(
    in_features=body.get_sentence_embedding_dimension(),
    out_features=num_classes,
    device=body.device,
)

model = SetFitModelWithEarlyStopping(
    model_body=body,
    model_head=head,
    normalize_embeddings=True,
)
model.to(body.device);

In [ ]:
args = EarlyStoppingTrainingArguments()
args.max_length = body.tokenizer.model_max_length

In [ ]:
model.fit(
    x_train=train_dataset["text"], y_train=train_dataset["label"],
    x_eval=val_dataset["text"], y_eval=val_dataset["label"],
    
    num_epochs=10,
    batch_size=16,
    body_learning_rate=args.body_classifier_learning_rate,
    head_learning_rate=args.head_learning_rate,
    l2_weight=args.l2_weight,
    
    max_length=body.tokenizer.model_max_length,
    
    show_progress_bar=True,
    end_to_end=True,
    
    # added early stopping arguments
    compute_metrics=compute_metrics,
    metric_for_best_model="macro_f1", # NOTE: must match one of the keys returned by `compute_metrics`
    early_stopping_patience=2,
    early_stopping_threshold=0.03,
    greater_is_better=True,
)

In [ ]:
# free GPU
model.to("cpu");
del model
import torch
torch.cuda.empty_cache()
# print(torch.cuda.memory_summary())

### with custom early-stopping trainer class

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
from setfit.modeling import SetFitHead
from src.finetuning.setfit_extensions.class_weights_head import (
    compute_class_weights,
    SetFitHeadWithClassWeights
)
from src.finetuning.setfit_extensions.early_stopping import (
    SetFitModelWithEarlyStopping, 
    EarlyStoppingTrainingArguments,
    EarlyStoppingCallback,
    SetFitEarlyStoppingTrainer
)

In [6]:
def model_init(
        model_name: str="sentence-transformers/all-MiniLM-L6-v2",
        num_classes: int=2, 
        class_weights: np._typing.NDArray=None,
        **kwargs
    ) -> SetFitModelWithEarlyStopping:
    
    model_kwargs={"device_map": "auto", **kwargs}
    body = SentenceTransformer(model_name, model_kwargs=model_kwargs, trust_remote_code=True)
    
    head_kwargs = dict(
        in_features=body.get_sentence_embedding_dimension(),
        out_features=num_classes,
        device=body.device,
    )
    head = SetFitHeadWithClassWeights(**head_kwargs, class_weights=class_weights) if class_weights is not None else SetFitHead(**head_kwargs)
    
    return SetFitModelWithEarlyStopping(
        model_body=body,
        model_head=head.to(body.device),
        normalize_embeddings=True,
    )

In [7]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"

In [8]:
from sentence_transformers.losses import ContrastiveLoss

training_args = EarlyStoppingTrainingArguments(
    num_epochs=(1, 15),
    batch_size=(16, 8),
    loss=ContrastiveLoss,
    # sentence transformer (embedding) finetuning arts
    eval_strategy="steps", # NOTE: currently no effect on (early stopping in) classification head training
    eval_steps=25, # NOTE: overwrites 0 epochs above for sentence transformer finetuning
    max_steps=200,
    eval_max_steps=200,
    # early stopping config
    metric_for_best_model=("embedding_loss", "f1"),
    greater_is_better=(False, True),
    load_best_model_at_end=True,
    save_total_limit=2, # NOTE: currently no effect on (early stopping in) classification head training
    # misc
    end_to_end=True,
)

training_callbacks = [
    EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.03), # for sentence transformer finetuning
    EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.02), # for classifier finetuning
]

In [9]:
# compute class weights (inversely proportional to class frequencies)
class_weights = compute_class_weights(train_dataset["label"])

In [10]:
# initialize Trainer
trainer = SetFitEarlyStoppingTrainer(
    model_init=lambda : model_init(
        model_name=model_id,
        num_classes=num_classes,
        class_weights=class_weights,
    ),
    metric="f1",
    metric_kwargs={"average": "macro"},
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=training_callbacks,
)
# fix max_length issue
trainer._args.max_length = trainer.st_trainer.model.tokenizer.model_max_length

# set seeds for reproducibility
trainer._args.seed = 42
trainer.st_trainer.args.seed = 42
trainer.st_trainer.args.data_seed = 42
trainer.st_trainer.args.full_determinism = True

# don't report to wandb or other experiment trackers
trainer._args.report_to = 'none'
trainer.st_trainer.args.report_to = 'none'

In [11]:
# train
trainer.train()

***** Running training *****
  Num unique pairs = 3200
  Batch size = 16
  Num epochs = 1


Step,Training Loss,Validation Loss
25,0.332100,0.063575
50,0.091700,0.030582
75,0.091700,0.027916
100,0.026500,0.030410
125,0.026500,0.033419
150,0.021000,0.031678


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1: calculating validation f1 for early stopping


PredictionOutput(predictions=array([[ 1.0363,  0.5545, -0.9856, -1.5584],
       [ 1.4341,  0.5075, -1.0535, -1.6227],
       [ 2.0253, -0.689 , -0.9648, -1.2838],
       [ 0.101 , -0.4059,  1.0925, -1.5835],
       [-0.7884, -1.7471, -0.1936,  2.3717],
       [ 0.3411,  2.4759, -1.2385, -2.0658],
       [-1.0845, -1.9056,  1.1285,  1.0459],
       [-1.747 , -2.0147,  1.5996,  1.5324],
       [ 0.1582,  2.7168, -1.2572, -2.0712],
       [ 0.9603,  2.1955, -1.6582, -2.1756],
       [-1.2239, -1.835 ,  0.8124,  1.7198],
       [ 0.4779,  2.4681, -1.3953, -1.9714],
       [-1.1894, -1.6443,  0.2355,  2.1521],
       [-1.1085, -2.562 ,  1.0953,  2.2751],
       [ 0.8485,  2.601 , -1.6191, -2.3294],
       [-0.9469, -1.7328,  1.7461,  0.124 ],
       [ 0.0911,  2.1297, -0.9583, -1.6805],
       [ 0.2401,  2.3346, -0.7257, -2.2802],
       [ 2.0911, -1.1618, -0.3338, -1.2866],
       [ 1.0353,  2.4069, -1.4579, -2.5371],
       [ 0.5432, -0.2233,  0.7707, -1.9382],
       [ 2.7358, -0.2713, 

...

{'training loss': 0.11004553354034821, 'validation loss': 0.5639995344728231, 'f1': 0.7969784901851449}


Epoch 5: calculating validation f1 for early stopping


PredictionOutput(predictions=array([[ 0.4651, -2.9064, -0.4172,  2.2319],
       [ 4.5518,  0.8038, -1.4967, -3.875 ],
       [-2.6658, -2.638 ,  2.7841,  1.6389],
       [-3.2118, -2.6212, -0.3314,  6.2702],
       [-1.5614,  5.2536, -1.8424, -1.5448],
       [-1.7182, -2.6278,  4.255 , -0.9065],
       [ 5.2128, -1.4697, -1.7751, -2.1904],
       [-2.8427, -2.3499,  4.1367, -0.0428],
       [ 1.2571,  2.3571, -2.1697, -2.2429],
       [ 0.4192,  5.148 , -2.6161, -2.982 ],
       [-1.2244,  4.2861, -1.8181, -1.154 ],
       [-0.3058,  4.6647, -2.846 , -1.7238],
       [-0.9098,  4.5495, -1.0066, -2.6474],
       [-3.9206, -3.1101,  1.9385,  4.7466],
       [-2.25  , -3.0904, -0.5003,  5.8228],
       [ 2.5266, -3.6461,  0.2063,  0.1533],
       [-1.3219, -3.3433,  0.6744,  3.187 ],
       [-2.6859, -3.3002,  0.0112,  6.0571],
       [-2.3557, -3.7249,  2.2814,  2.7283],
       [-3.0409, -2.8798,  0.212 ,  5.8206],
       [ 1.2235,  3.92  , -2.8289, -2.5111],
       [-1.9495, -2.4667, 

Epoch:  27%|██▋       | 4/15 [00:19<00:53,  4.85s/it]

{'training loss': 0.05586615695307652, 'validation loss': 0.6810536684468389, 'f1': 0.7881568239730163}
Early stopping triggered after 5 epochs
Loading best model from epoch 1


In [12]:
# verify best model loaded
trainer.evaluate(val_dataset)

***** Running evaluation *****


{'f1': 0.8080587999672901}

In [13]:
# eval
trainer.evaluate(test_dataset, "test")

***** Running evaluation *****


{'f1': 0.7934602596680194}

In [15]:
from sklearn.metrics import classification_report
preds = trainer.model.predict(test_dataset["text"], as_numpy=True)
print(classification_report(test_dataset['label'], preds, target_names=train_dataset.features['label'].names))

              precision    recall  f1-score   support

       World       0.81      0.83      0.82        42
      Sports       0.93      0.94      0.94        70
    Business       0.91      0.49      0.64        43
    Sci/Tech       0.67      0.93      0.78        45

    accuracy                           0.82       200
   macro avg       0.83      0.80      0.79       200
weighted avg       0.84      0.82      0.81       200



## Multi-label classification

In [16]:
from datasets import load_dataset
from collections import Counter

dataset_id = 'acloudfan/toxicity-multi-label-classifier'

train_dataset = load_dataset(dataset_id, split="train")
val_dataset = load_dataset(dataset_id, split="validation")
test_dataset = load_dataset(dataset_id, split="test")

label_cols = ['toxic', 'threat', 'insult', 'identity_hate']
for col in label_cols:
    print(col, dict(Counter(train_dataset[col])), sep=": ")

num_classes = len(label_cols)
id2label = dict(enumerate(label_cols))
label2id = {v: k for k, v in id2label.items()}

# convert to multi-label format
def format_dataset_multi_label(example):
    example['label'] = [example[label_col] for label_col in label2id.keys()]
    return example

train_dataset = train_dataset.map(format_dataset_multi_label, batched=False)
train_dataset = train_dataset.rename_column("comment_text", "text")
train_dataset = train_dataset.remove_columns(label_cols)

val_dataset = val_dataset.map(format_dataset_multi_label, batched=False)
val_dataset = val_dataset.rename_column("comment_text", "text")
val_dataset = val_dataset.remove_columns(label_cols)

test_dataset = test_dataset.map(format_dataset_multi_label, batched=False)
test_dataset = test_dataset.rename_column("comment_text", "text")
test_dataset = test_dataset.remove_columns(label_cols)

toxic: {1: 58, 0: 31}
threat: {0: 69, 1: 20}
insult: {1: 26, 0: 63}
identity_hate: {0: 69, 1: 20}


In [17]:
import numpy as np
import torch
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)

def compute_metrics_multilabel(p):
    """
    Compute evaluation metrics for multi-label classification.
    eval_pred: transformers.trainer_utils.PredictionOutput
               (contains .predictions and .label_ids)
    """
    # unpack
    logits, labels = p.predictions, p.label_ids
    # apply sigmoid to get probabilities in [0,1]
    #probs = 1 / (1 + np.exp(-logits))
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # threshold at 0.5 for binary decisions
    preds = (probs > 0.5).astype(int)

    # compute metrics
    accuracy = accuracy_score(labels, preds)
    f1_macro = f1_score(labels, preds, average="macro", zero_division=0.0)
    f1_micro = f1_score(labels, preds, average="micro", zero_division=0.0)
    precision_macro = precision_score(labels, preds, average="macro", zero_division=0.0)
    recall_macro = recall_score(labels, preds, average="macro", zero_division=0.0)

    # optional: subset accuracy (exact match ratio)
    subset_acc = (labels == preds).all(axis=1).mean()

    return {
        "f1_macro": f1_macro,
        # "f1_micro": f1_micro,
        # "precision_macro": precision_macro,
        # "recall_macro": recall_macro,
        "accuracy": accuracy,
        # "subset_accuracy": subset_acc,
    }

# test:
# y_true = train_dataset['label'][:10]
# # simulate some predictions by sampling (shape (10, num_classes)) uniformly from 0-1
# np.random.seed(42)
# y_pred = np.random.rand(10, num_classes)
# p = PredictionOutput(predictions=y_pred, label_ids=y_true, metrics=None)
# compute_metrics_multilabel(p)

### simple `fit` method

In [ ]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
body = SentenceTransformer(model_id, model_kwargs={"device_map": "auto"})

head = SetFitHead(
    in_features=body.get_sentence_embedding_dimension(),
    out_features=num_classes,
    device=body.device,
    multitarget=True,
)

model = SetFitModelWithEarlyStopping(
    model_body=body,
    model_head=head,
    multi_target_strategy="one-vs-rest",
    normalize_embeddings=True,
)
model.to(body.device);

In [ ]:
args = EarlyStoppingTrainingArguments()
args.max_length = body.tokenizer.model_max_length

model.fit(
    x_train=train_dataset["text"], y_train=train_dataset["label"],
    x_eval=val_dataset["text"], y_eval=val_dataset["label"],
    
    num_epochs=30,
    batch_size=16,
    body_learning_rate=args.body_classifier_learning_rate,
    head_learning_rate=args.head_learning_rate,
    l2_weight=args.l2_weight,
    
    max_length=args.max_length,
    
    show_progress_bar=True,
    end_to_end=True,
    
    # added early stopping arguments
    compute_metrics=compute_metrics_multilabel,
    metric_for_best_model="f1_macro", # NOTE: must match one of the keys returned by `compute_metrics`
    early_stopping_patience=4,
    early_stopping_threshold=0.03,
    greater_is_better=True,
)

In [ ]:
# verify best model loaded
logits = model.predict_logits(val_dataset["text"], as_numpy=True)
p = PredictionOutput(predictions=logits, label_ids=np.array(val_dataset["label"]), metrics={})
compute_metrics_multilabel(p)

### with trainer

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from setfit.modeling import SetFitHead
from src.finetuning.setfit_extensions.class_weights_head import (
    compute_class_weights,
    SetFitHeadWithClassWeights
)
from src.finetuning.setfit_extensions.early_stopping import (
    SetFitModelWithEarlyStopping, 
    EarlyStoppingTrainingArguments,
    EarlyStoppingCallback,
    SetFitEarlyStoppingTrainer
)

In [19]:
from typing import Literal
def multiclass_model_init(
        model_name: str="sentence-transformers/all-MiniLM-L6-v2",
        num_classes: int=2, 
        multi_target_strategy: Literal["one-vs-rest", "multi-output"]="one-vs-rest",
        class_weights: np._typing.NDArray=None,
        **kwargs
    ) -> SetFitModelWithEarlyStopping:
    
    model_kwargs={"device_map": "auto", **kwargs}
    body = SentenceTransformer(model_name, model_kwargs=model_kwargs, trust_remote_code=True)
    
    head_kwargs = dict(
        in_features=body.get_sentence_embedding_dimension(),
        out_features=num_classes,
        device=body.device,
        multitarget=(multi_target_strategy is not None),
    )
    head = SetFitHeadWithClassWeights(**head_kwargs, class_weights=class_weights) if class_weights is not None else SetFitHead(**head_kwargs)
    
    return SetFitModelWithEarlyStopping(
        model_body=body,
        model_head=head.to(body.device),
        multi_target_strategy=multi_target_strategy,
        normalize_embeddings=True,
    )

In [20]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"

In [21]:
training_args = EarlyStoppingTrainingArguments(
    num_epochs=(1, 15),
    batch_size=(16, 8),
    loss=ContrastiveLoss,
    # sentence transformer (embedding) finetuning arts
    eval_strategy="steps", # NOTE: currently no effect on (early stopping in) classification head training
    eval_steps=25, # NOTE: overwrites 0 epochs above for sentence transformer finetuning
    max_steps=200,
    eval_max_steps=200,
    # train end to end
    end_to_end=True,
    # early stopping config
    metric_for_best_model=("embedding_loss", "f1"),
    greater_is_better=(False, True),
    load_best_model_at_end=True,
    save_total_limit=2, # NOTE: currently no effect on (early stopping in) classification head training
)

In [22]:
# compute class weights (inversely proportional to class frequencies)
class_weights = compute_class_weights(train_dataset["label"], multitarget=True)
class_weights

array([0.1107, 0.3211, 0.247 , 0.3211])

In [23]:
training_callbacks = [
    EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.03), # for sentence transformer finetuning
    EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.02), # for classifier finetuning
]

# initialize Trainer
trainer = SetFitEarlyStoppingTrainer(
    model_init=lambda : multiclass_model_init(
        model_name=model_id,
        num_classes=num_classes,
        # class_weights=class_weights,
    ),
    args=training_args,
    metric="f1",
    metric_kwargs={"average": "macro"},
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=training_callbacks,
    # compute_metrics=compute_metrics_multilabel,
)
# fix max_length issue
trainer._args.max_length = trainer.st_trainer.model.tokenizer.model_max_length

# set seeds for reproducibility
trainer._args.seed = 42
trainer.st_trainer.args.seed = 42
trainer.st_trainer.args.data_seed = 42
trainer.st_trainer.args.full_determinism = True

# don't report to wandb or other experiment trackers
trainer._args.report_to = 'none'
trainer.st_trainer.args.report_to = 'none'

In [24]:
# train
trainer.train()

***** Running training *****
  Num unique pairs = 3200
  Batch size = 16
  Num epochs = 1


Step,Training Loss,Validation Loss
25,0.236100,0.056776
50,0.077500,0.028888
75,0.077500,0.025364
100,0.016700,0.024898
125,0.016700,0.024293
150,0.012600,0.023766


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1: calculating validation f1 for early stopping


Epoch:   7%|▋         | 1/15 [00:00<00:07,  1.80it/s]

...

Epoch:  87%|████████▋ | 13/15 [00:06<00:00,  2.03it/s]

PredictionOutput(predictions=array([[ 4.6856, -0.944 ,  1.469 ,  0.0815],
       [ 3.7221, -0.782 ,  0.3766, -1.4167],
       [ 4.2925,  0.6515, -0.8282, -1.3382],
       [-3.1899, -3.316 , -2.6113, -2.2385],
       [ 3.2051,  2.447 , -1.6613, -1.7003],
       [ 4.3126,  0.6924, -0.6971, -0.7543],
       [-1.102 , -2.6558, -0.7123, -0.7475],
       [-2.5083, -2.7747, -3.4263, -2.3512],
       [ 0.1988, -2.817 , -2.2702, -2.1261],
       [ 1.7471, -0.1773, -2.4413, -2.0876],
       [-1.8426, -2.6169, -2.2073, -1.1423],
       [-3.0131, -2.1529, -2.0396, -1.7605],
       [ 2.5297, -1.6552, -0.0066, -0.947 ],
       [ 2.4337, -0.9297, -1.4851, -1.9815],
       [ 2.141 , -2.3689, -1.0365, -0.4455],
       [ 2.6345, -1.8495, -0.7458, -0.8748],
       [ 0.37  , -0.7393, -2.6742, -1.5852],
       [ 0.132 , -2.4257, -2.1234, -1.5597],
       [ 2.7434,  0.498 , -1.2077, -0.98  ],
       [ 4.0627,  0.2539, -0.7934, -0.4652],
       [ 2.9178, -1.3472, -0.1878, -1.3837],
       [-2.5496, -1.9306, 

Epoch 14: calculating validation f1 for early stopping


Epoch:  87%|████████▋ | 13/15 [00:06<00:01,  1.89it/s]

PredictionOutput(predictions=array([[ 1.8944, -0.1184, -2.5087, -2.1723],
       [ 3.0425, -1.3867, -0.1459, -1.4549],
       [ 3.3009,  2.5905, -1.711 , -1.7854],
       [ 2.5952, -1.5705, -0.9283, -1.6287],
       [ 1.9191,  2.134 , -2.2825, -1.8574],
       [ 2.3022, -2.4566, -1.0404, -0.4397],
       [-3.0231, -2.1914, -2.0581, -1.802 ],
       [ 3.2879,  0.6717, -1.2762, -1.7177],
       [-2.5521, -1.963 , -2.2043, -2.4056],
       [ 3.7307, -0.7326, -1.1563, -0.4375],
       [ 1.2385, -2.132 , -2.8391, -1.4558],
       [ 2.7921, -1.906 , -0.7337, -0.889 ],
       [ 4.4278,  1.481 , -0.7691, -1.5059],
       [ 4.2645,  0.3742, -0.7884, -0.449 ],
       [ 2.6112, -0.9281, -1.7196, -2.0295],
       [-1.6771, -1.9956, -1.1973, -1.3308],
       [-3.1615, -3.3996, -2.6467, -2.2929],
       [ 3.9067,  0.1629, -1.0416, -0.0792],
       [-2.4505, -2.8303, -3.5163, -2.4155],
       [ 4.282 , -1.9923,  2.0934,  0.3564],
       [ 0.0622, -2.0455, -2.67  , -2.9986],
       [ 2.8766,  0.5498, 

In [25]:
trainer.evaluate(val_dataset, metric_key_prefix="eval")

***** Running evaluation *****


{'f1': 0.6216783216783217}

In [ ]:
from sklearn.metrics import classification_report
preds = trainer.model.predict(test_dataset['text'], as_numpy=True)
print(classification_report(test_dataset['label'], preds, zero_division=0, target_names=label_cols))

               precision    recall  f1-score   support

        toxic       0.88      0.92      0.90        25
       threat       0.86      0.75      0.80         8
       insult       0.67      0.20      0.31        10
identity_hate       0.00      0.00      0.00         9

    micro avg       0.86      0.60      0.70        52
    macro avg       0.60      0.47      0.50        52
 weighted avg       0.69      0.60      0.62        52
  samples avg       0.64      0.48      0.52        52



: 